In [1]:
import sys, glob, subprocess
from itertools import product

import shutil
#from IPython.display import Image as JImage

import numpy as np
import cv2
import dlib
from PIL import Image
from skimage import io
from matplotlib.gridspec import GridSpec
from matplotlib import pyplot as plt

import tensorflow as tf

import keras
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input, decode_predictions

Using TensorFlow backend.


In [30]:
def detect_faces(image):
  
    #cnn detector사용해서 얼굴 인식하는 것
    cnn_face_detector=dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")
    detected_faces=cnn_face_detector(img,1)
    face_frames=[(x.rect.left(),x.rect.top(),x.rect.right(),x.rect.bottom()) for x in detected_faces]
    
    return face_frames

In [2]:
def realtime(model_path): #실시간으로 딥러닝 모델 동작하는 함수 , 작동은 잘되는데, 너무 특정 각도에서만 동작
    def label_img(img, label, loc=(3,50)):
        return cv2.putText(img, label, loc, cv2.FONT_HERSHEY_SIMPLEX, 3.4, (0, 255, 0), 2, cv2.LINE_AA)
#cv2.putText는 웹캠 영상에다가 글쓰는거    
    
    """     
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# 얼굴인지 아닌지를 분류해줌 카스케이드 파일
# 지금은 이 파일 대신 얼굴을 분류해주는 함수를 따로 만들어서 그걸 동작
# 이 함수에서는 아직 안씀, 얼굴인지 분류 안해주고 일단 눈 떴는지 감았는지만
    """  
    classes={1:"open",0:"close"} #classification 작업이기 때문에 0아니면 1
    model=keras.models.load_model(model_path) #h5 가중치 파일 불러오기
    
    cap=cv2.VideoCapture(0) #웹캠으로부터 비디오 읽기 - 0이라는게 웹캠 카메라를 의미함
    cap.open(0) #웹캠 열기
    cap.set(3,640) #웹캠 사이즈 설정 - 3은 너비 
    cap.set(4,480) #웹캠 사이즈 설정 - 4는 높이
    
    #noface=list()
    while(True) : #인터럽트가 일어나야 종료가 되므로 그전까지 계속 웹캠 작동
        images=list() #순간순간 캡처를 리스트로 저장
        code,image = cap.read() #code는 capture결과 (boolean 타입), image는 capure한 frame
        if code: #만약 capture 결과가 true라면 (즉 capture한게 있다면)
            images.append(image) #리스트에 캡처한 이미지를 추가
            #ace_rect=detect_faces(image)
            #if len(face_rect)>0:
                #mage=np.array(Image.fromarray(image).crop(face_rect[0]))
            #lse:
                #noface.append(i)
            
#우리는 데이터 224x224로 학습시켰기 때문에 영상도 224x224사이즈로 전처리     
        processed_images=np.stack([cv2.resize(preprocess_input(img.astype(np.float32)),(224,224)) for img in images], axis=0)

        
        preds=model.predict(processed_images)
        labels=[classes[p] for p in np.argmax(preds, axis=1)] #close/open label for each frame
        #labels=["noface" if i in noface else label for i,label in enumerate(labels)]
        for j,image in enumerate(images):
            which=0 if labels[j]=="close" else 1
            label=labels[j] + " (%0.2f)" %(preds[j][which])
            img_a=label_img(image, label) #annotate each original frame with predicted label
            
          #화면 제목 설정
        cv2.imshow('frame',image)
        
        #q클릭하면 웹캠 정상 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.waitKey(0)
    cv2.destoryAllWindows()
    return
    
    
model_path="FINAL1.h5" # 모델경로
    

In [3]:
realtime(model_path) #위에서 정의한 함수 실행

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




AttributeError: module 'cv2.cv2' has no attribute 'destoryAllWindows'

In [ ]:
def detect_faces(image):
  #이거는 dlib라이브러리 사용해서 얼굴임을 인식하는 함수
    face_detector=dlib.get_frontal_face_detector()
    detected_faces=face_detector(image,1)
    face_frames=[(x.left(),x.top(),x.right(),x.bottom()) for x in detected_faces]
    
   
    
    return face_frames

In [25]:
def realtime2(model_path): #이건 실시간으로 얼굴인식 + 딥러닝으로 눈 구별 하려고하는 함순데 구현 못하겠음
    def label_img(img, label, loc=(3,50)): #annotates the image with the predicted label (close/open)
        return cv2.putText(img, label, loc, cv2.FONT_HERSHEY_SIMPLEX, 3.4, (0, 255, 0), 2, cv2.LINE_AA)
    
    cap=cv2.VideoCapture(0)
    cap.set(3,640)
    cap.set(4,480)
    
    
    
    classes={1:"open",0:"close"} #classification 작업이기 때문에 0아니면 1
    model=keras.models.load_model(model_path) #h5 가중치 파일 불러오기
    
    
    noface=list()
    while(True) : #인터럽트가 일어나야 종료가 되므로 그전까지 계속 웹캠 작동
        images=list() #순간순간 캡처를 리스트로 저장
        code,image = cap.read() #code는 capture결과 (boolean 타입), image는 capure한 frame
        if code: #만약 capture 결과가 true라면 (즉 capture한게 있다면)
            
            
            face_rect=detect_faces(image)# detect_faces함수에 캡처한 이미지를 매개변수로
            if len(face_rect)>0: #face가 하나라도 있으면
                image=np.array(Image.fromarray(image).crop(face_rect[0]))
            else:
                noface.append(cap.get(cv2.CAP_PROP_FPS))
                
            images.append(image) #리스트에 캡처한 이미지를 추가
            
            
#우리는 데이터 224x224로 학습시켰기 때문에 영상도 224x224사이즈로 전처리     
        processed_images=np.stack([cv2.resize(preprocess_input(img.astype(np.float32)),(224,224)) for img in images], axis=0)

        
        preds=model.predict(processed_images)
        labels=[classes[p] for p in np.argmax(preds, axis=1)] #close/open label for each frame
        labels=["noface" if i in noface else label for i,label in enumerate(labels)]
        for j,image in enumerate(images):
            which=0 if labels[j]=="close" else 1
            label=labels[j] + (" (%0.2f)" %(preds[j][which]) if label in classes.keys() else '')
            img_a=label_img(image, label) #annotate each original frame with predicted label
            
          #화면 제목 설정
        cv2.imshow('frame',image)
        
        #q클릭하면 웹캠 정상 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return;
    
    

In [27]:
realtime2(model_path)

UnboundLocalError: local variable 'label' referenced before assignment